In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**According to Winston Churchill, " We shape our homes, then our homes shape us "**

Hence, in this notebook let's predict the house price using XGBoost.

**IMPORTING LIBRARIES**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error


**To load the training and validation sets in X_train, X_valid, y_train, and y_valid. The test set is loaded in X_test.**

In [ ]:
# Read the data
X = pd.read_csv('/kaggle/input/home-data-for-ml-course/train.csv')
X_test_full = pd.read_csv('/kaggle/input/home-data-for-ml-course/test.csv')

# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice              
X.drop(['SalePrice'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numeric columns
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

# One-hot encode the data (to shorten the code, we use pandas)
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

**XGBOOST - Extreme Gradient Boosting**

XGBOOST is an implementation of gradient boosting with several additional features focused on performance and speed.



**Build and train the model**

By defining the model with my_model_1 = XGBRegressor(random_state=0). Then, we can fit the model with the fit() method.



In [ ]:
# Define the model
model_1 = XGBRegressor(random_state=0) # Your code here

# Fit the model
model_1.fit(X_train, y_train) # Your code here

**Set `predictions_1` to the model's predictions for the validation data.**

 Using the predict() method to generate validation predictions.



In [ ]:
# Get predictions
predictions_1 = model_1.predict(X_valid)

**Finally, using the mean_absolute_error() function to calculate the mean absolute error (MAE) corresponding to the predictions for the validation set. Note that the labels for the validation data are stored in y_valid.**



In [ ]:
# Calculate MAE
mae_1 = mean_absolute_error(predictions_1, y_valid)
print("Mean Absolute Error:" , mae_1)

**Improving the model**

Initially we trained a default model as baseline, it's time to tinker with the parameters, to see if we can get better performance!
According to this problem, we chose to increase the number of trees in the model (with the n_estimators parameter) and decrease the learning rate (with the learning_rate parameter).

In [ ]:
# Define the model
model_2 = XGBRegressor(n_estimators=1000, learning_rate=0.05)

# Fit the model
model_2.fit(X_train, y_train)

# Get predictions
predictions_2 = model_2.predict(X_valid)

# Calculate MAE
mae_2 = mean_absolute_error(predictions_2, y_valid)
print("Mean Absolute Error:" , mae_2)

**Hence, our model in model_2 has lower MAE than the model in model_1.**

In [ ]:
test_preds = model_2.predict(X_test)

In [ ]:
output = pd.DataFrame({'Id': X_test_full.Id,'SalePrice': test_preds})
output.to_csv('submission.csv', index=False)
print("Successfully Predicted ✍ the 🏡💒🏠 Price using XGBoost✨💰")

In [ ]:
from IPython.display import Image
import os

Image('/kaggle/input/home-sweet-home-pic/home_sweet_home.jpg')

**According to Phillip Moffitt, "A house is a home when it shelters the body & comforts the soul"**

Hence, in this notebook we have successfully predicted the house price using XGBoost.